In [1]:
# Necessary imports
%load_ext autoreload
%autoreload 2\
    
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from definitions import ROOT_DIR



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import pandas as pd
import numpy as np

In [3]:
from src.features.multi_omics import MultiOmicsData

lusc_data = MultiOmicsData(cancer_type="LUSC", 
                           folder_path="/home/jonny2/PycharmProjects/assn-miRNA-LUAD/data/tcga-assembler/LUSC/", 
                           modalities=["GE", "MIR"])

/opt/anaconda/lib/python2.7/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


('DRUGS', (357, 4))
('PATIENTS', (504, 5))
('MIR', (380, 1870))
('GE', (552, 20472))


In [ ]:
centroids = pd.read_csv("/home/jonny2/PycharmProjects/assn-miRNA-LUAD/data/external/wilkerson.scc/predictor.centroids.csv")

In [ ]:
centroids.columns = ["genes", "primitive", "classical", "secretory", "basal"]

In [ ]:
centroids = centroids[centroids["genes"].isin(lusc_data.GE.get_genes_list())]
genes = centroids["genes"]

In [ ]:
centroids.index = centroids.genes
centroids.drop(['genes'], axis=1, inplace=True)
centroids = centroids.T
centroids

# Subsetting the GE data to only genes

In [ ]:
df = lusc_data.GE.data[genes]

In [ ]:
# Normalize 
df_norm = (df.T - df.T.mean()) / (df.T.max() - df.T.min())
df_norm = df_norm.T

In [ ]:
df = df[df.index.str.contains("-01A")]

# Classify LUSC patients based on cluster centroids obtained from Wilkenson, et. al

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4)

In [ ]:
df = centroids.values
centroid_values = (df.T - df.T.mean()) / (df.T.max() - df.T.min()).T
kmeans.cluster_centers_ = centroid_values.T

In [ ]:
kmeans.cluster_centers_.shape

In [ ]:
kmeans.predict(df)

In [ ]:
len(kmeans.predict(df))

In [ ]:
pd.Series(kmeans.predict(df)).value_counts()
#primitive 7%, classical 45%, secretory 13%, basal 34%

In [ ]:
pd.DataFrame(kmeans.predict(df), index=df.index)
